# Automate algo trading with Python and IB

In today’s issue, I’m going to show you how to send a trade order to Interactive Brokers (IB) with Python.

IB is a US-based broker that has an API that lets traders send trade orders through algorithms. IB has cheap commissions, access to global markets, and good execution. I recommend using IB for trading, both manually and algorithmically.

Some traders use the API to automate part of their trading. Some build sophisticated algorithms for fully-automated trading systems. Others use the API for storing real-time market data.

If you don’t have an IB set up yet, head over to Interactive Brokers and set one up. If you use this link, you can earn up to $1,000.

By the end of this issue, you’ll know how to:

Install the Python API
Install Trader Workstation
Connect to IB through Python

In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper

class IBapi(EWrapper, EClient):
     def __init__(self):
         EClient.__init__(self, self)

app = IBapi()
app.connect("127.0.0.1", 7497, 123)
app.run()

# uncomment this section if unable to connect
# import time
# time.sleep(2)
# app.disconnect()

In [ ]:
# Step 2: Buy AAPL

# To buy a stock, you send information about what you want to buy to IB through the API. The app.placeOrder the method does all the hard work for you.

# First, import the classes from the IB API. Import threading to run the app in a single thread and time to pause execution of the script.

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import threading
import time
# This class creates the app. The app is what you use to interact with IB.

# This class inherits two classes from the IB API. If this doesn’t make sense, don’t worry. The API needs an order ID associated with every order. nextValidId is a built-in function that finds the next available order ID. The other methods log output so you can see what’s happening.

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print("The next valid order id is: ", self.nextorderId)

    def orderStatus(
        self,
        orderId,
        status,
        filled,
        remaining,
        avgFullPrice,
        permId,
        parentId,
        lastFillPrice,
        clientId,
        whyHeld,
        mktCapPrice,
    ):
        print(
            "orderStatus - orderid:",
            orderId,
            "status:",
            status,
            "filled",
            filled,
            "remaining",
            remaining,
            "lastFillPrice",
            lastFillPrice,
        )

    def openOrder(self, orderId, contract, order, orderState):
        print(
            "openOrder id:",
            orderId,
            contract.symbol,
            contract.secType,
            "@",
            contract.exchange,
            ":",
            order.action,
            order.orderType,
            order.totalQuantity,
            orderState.status,
        )

    def execDetails(self, reqId, contract, execution):
        print(
            "Order Executed: ",
            reqId,
            contract.symbol,
            contract.secType,
            contract.currency,
            execution.execId,
            execution.orderId,
            execution.shares,
            execution.lastLiquidity,
        )

In [ ]:
# Next, create a function to run the app. Then, define a stock contract (IB calls everything a contract).

def run_loop():
    app.run()

def stock_contract(
    symbol,
    secType='STK',
    exchange='SMART',
    currency='USD'
):
    # create a stock contract
    contract = Contract()
    contract.symbol = symbol
    contract.secType = secType
    contract.exchange = exchange
    contract.currency = currency

    return contract

In [ ]:
# Now that the setup is out of the way, make the connection and start a thread. The while loop checks if the API is connected. If it is, app.nextorderId returns an int . Otherwise, it returns None.

app = IBapi()
app.connect('127.0.0.1', 7497, 123)

app.nextorderId = None

api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

while True:
    if isinstance(app.nextorderId, int):
        print('connected')
        break
    else:
        print('waiting for connection')
        time.sleep(1)

In [ ]:
order = Order()
order.action = "BUY"
order.totalQuantity = 10
order.orderType = "LMT"
order.lmtPrice = "130.00"

app.placeOrder(app.nextorderId, stock_contract("AAPL"), order)

# uncomment if you are sending more than one order
# app.nextorderId += 1

time.sleep(3)

# cancel the order for the demo
print('cancelling order')
app.cancelOrder(app.nextorderId, "")

time.sleep(3)
app.disconnect()